# The BSSN Formulation of General Relativity in Generic Curvilinear Coordinates

<font color='green'>**All expressions generated in this module have been validated against a trusted code (the original NRPy+/SENR code, which itself was validated against [Baumgarte's code](https://arxiv.org/abs/1211.6632)).**</font>

NRPy+'s original purpose was to be an easy-to-use code capable of generating Einstein's equations in a broad class of [singular](https://en.wikipedia.org/wiki/Coordinate_singularity), curvilinear coordinate systems, where the user need only input the scale factors of the underlying reference metric. Upon generating these equations, NRPy+ would then leverage SymPy's [common-expression-elimination (CSE)](https://en.wikipedia.org/wiki/Common_subexpression_elimination) and C code generation routines, coupled to its own [single-instruction, multiple-data (SIMD)](https://en.wikipedia.org/wiki/SIMD) functions, to generate highly-optimized C code.

This tutorial module demonstrates how Einstein's equations of general relativity in this formulation are constructed and output within NRPy+. As Einstein's equations in this formalism take the form of highly nonlinear, coupled *wave equations*, the [tutorial module on the scalar wave equation in curvilinear coordinates](Tutorial-ScalarWaveCurvilinear.ipynb) is *required* reading before beginning this module. That module, as well as its own prerequisite [module on reference metrics within NRPy+](Tutorial-Reference_Metric.ipynb) provides the needed overview of how NRPy+ handles reference metrics.

## Background Reading/Lectures

* Mathematical foundations of BSSN and 3+1 initial value problem decompositions of Einstein's equations:
    * [Thomas Baumgarte's lectures on mathematical formulation of numerical relativity](https://www.youtube.com/watch?v=t3uo2R-yu4o&list=PLRVOWML3TL_djTd_nsTlq5aJjJET42Qke)
    * [Yuichiro Sekiguchi's introduction to BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) 
* Extensions to the standard BSSN approach used in NRPy+
    * [Brown's covariant "Lagrangian" formalism of BSSN](https://arxiv.org/abs/0902.3652)
    * [BSSN in spherical coordinates, using the reference-metric approach of Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632)
    * [BSSN in generic curvilinear coordinates, using the extended reference-metric approach of Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)

## Python module containing the final expressions constructed in this module: [BSSN/BSSN_RHS.py](../edit/BSSN/BSSN_RHSs.py)

## A Note on Notation

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.

As a corollary, any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one: A Latin index in a four-vector will be incremented and a Greek index in a three-vector will be decremented (however, the latter case does not occur in this tutorial module).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module documents and constructs the time evolution equations of the BSSN formulation of Einstein's equations, as defined  in [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658) (see also [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632)).

The module is organized as follows

1. [Step 1](#brownslagrangebssn): Brown's covariant formulation of BSSN (excluding the gauge conditions; see next section)
    1. [Step 1.a](#fullequations): Numerical implementation of BSSN time-evolution equations
        1. [Step 1.a.i](#liederivs): Expanding the Lie derivatives; the BSSN time-evolution equations in their final form
1. [Step 2](#gaugeconditions): Time-evolution equations for the gauge quantities $\alpha$ and $\beta^i$
1. [Step 3](#constraintequations): The BSSN constraint equations
    1. [Step 3.a](#hamiltonianconstraint): The Hamiltonian constraint
    1. [Step 3.b](#momentumconstraint): The momentum constraint

<a id='brownslagrangebssn'></a>

# Step 1: [Brown](https://arxiv.org/abs/0902.3652)'s covariant formulation of BSSN
$$\label{brownslagrangebssn}$$

The covariant "Lagrangian" BSSN formulation of [Brown (2009)](https://arxiv.org/abs/0902.3652), which requires

$$
\partial_t \bar{\gamma} = 0,
$$

results in the BSSN equations taking the following form (Eqs. 11 and 12 in [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)):

\begin{align}
  \partial_{\perp} \bar{\gamma}_{i j} {} = {} & \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_{\perp} \bar{A}_{i j} {} = {} & -\frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi \right . \nonumber \\
    & \left . + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_{\perp} \phi {} = {} & \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{\perp} K {} = {} & \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} \nonumber \\
  & - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_{\perp} \bar{\Lambda}^{i} {} = {} & \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \bar{A}^{j k} \Delta_{j k}^{i} \nonumber \\
  & -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\end{align}
where 
* the $\text{TF}$ superscript denotes the trace-free part.
* $\bar{\gamma}_{ij} = \varepsilon_{i j} + \hat{\gamma}_{ij}$, where $\bar{\gamma}_{ij} = e^{-4\phi} \gamma_{ij}$ is the conformal metric, $\gamma_{ij}$ is the physical metric (see below), and $\varepsilon_{i j}$ encodes information about the non-hatted metric.
* $\gamma_{ij}$, $\beta^i$, and $\alpha$ are the physical (as opposed to conformal) spatial 3-metric, shift vector, and lapse, respectively, which may be defined via the 3+1 decomposition line element (in [$G=c=1$ units](https://en.wikipedia.org/wiki/Planck_units)):
$$ds^2 = -\alpha^2 dt^2 + \gamma_{ij}\left(dx^i + \beta^i dt\right)\left(dx^j + \beta^j dt\right).$$
* $\bar{R}_{ij}$ is the conformal Ricci tensor, computed via
\begin{align}
  \bar{R}_{i j} {} = {} & - \frac{1}{2} \bar{\gamma}^{k l} \hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} + \bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k} + \Delta^{k} \Delta_{(i j) k} \nonumber \\
  & + \bar{\gamma}^{k l} \left (2 \Delta_{k(i}^{m} \Delta_{j) m l} + \Delta_{i k}^{m} \Delta_{m j l} \right ) \; .
\end{align}
* $\partial_{\perp} = \partial_t - \mathcal{L}_\beta$; $\mathcal{L}_\beta$ is the [Lie derivative](https://en.wikipedia.org/wiki/Lie_derivative) along the shift vector $\beta^i$.
* $\partial_0 = \partial_t - \beta^i \partial_i$ is an advective time derivative.
* $\hat{D}_j$ is the [covariant derivative](https://en.wikipedia.org/wiki/Covariant_derivative) with respect to the reference metric $\hat{\gamma}_{ij}$.
* $\bar{D}_j$ is the [covariant derivative](https://en.wikipedia.org/wiki/Covariant_derivative) with respect to the barred spatial 3-metric $\bar{\gamma}_{ij}$
* $\Delta^i_{jk}$ is the tensor constructed from the difference of barred and hatted Christoffel symbols:
$$\Delta^i_{jk} = \bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}$$
    * The related quantity $\Delta^i$ is defined $\Delta^i \equiv \bar{\gamma}^{jk} \Delta^i_{jk}$.
* $\bar{A}_{ij}$ is the conformal, trace-free extrinsic curvature: 
$$\bar{A}_{ij} = e^{-4\phi} \left(K_{ij} - \frac{1}{3}\gamma_{ij} K\right),$$
where $K$ is the trace of the extrinsic curvature $K_{ij}$.

<a id='fullequations'></a>

## Step 1.a: Numerical implementation of BSSN time-evolution equations
$$\label{fullequations}$$

Regarding the numerical implementation of the above equations, first notice the left-hand sides of the equations include the time derivatives. Numerically, these equations are solved using as an [initial value problem](https://en.wikipedia.org/wiki/Initial_value_problem), where data are specified at a given time $t$, so that the solution at any later time can be obtained using the [Method of Lines (MoL)](https://en.wikipedia.org/wiki/Method_of_lines). MoL requires that the equations be written in the form:

$$\partial_t \vec{U} = \vec{f}\left(\vec{U},\partial_i \vec{U}, \partial_i \partial_j \vec{U},...\right),$$

for the vector of "evolved quantities" $\vec{U}$, where the right-hand side vector $\vec{f}$ *does not* contain *explicit* time derivatives of $\vec{U}$.

Thus we must first rewrite the above equations so that *only* partial derivatives of time appear on the left-hand sides of the equations, meaning that the Lie derivative terms must be moved to the right-hand sides of the equations.

<a id='liederivs'></a>

### Step 1.a.i: Expanding the Lie derivatives; BSSN equations in their final form
$$\label{liederivs}$$

In this Step, we provide explicit expressions for the [Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative) $\mathcal{L}_\beta$ appearing inside the $\partial_\perp = \partial_t - \mathcal{L}_\beta$ operators for $\left\{\bar{\gamma}_{i j},\bar{A}_{i j},W, K, \bar{\Lambda}^{i}\right\}$.

In short, the Lie derivative of tensor weight $w$ is given by (from [the wikipedia article on Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative))
\begin{align}
(\mathcal {L}_X T) ^{a_1 \ldots a_r}{}_{b_1 \ldots b_s} &= X^c(\partial_c T^{a_1 \ldots a_r}{}_{b_1 \ldots b_s}) \\
&\quad - (\partial_c X ^{a_1}) T ^{c a_2 \ldots a_r}{}_{b_1 \ldots b_s} - \ldots - (\partial_c X^{a_r}) T ^{a_1 \ldots a_{r-1}c}{}_{b_1 \ldots b_s} \\
 &\quad  +  (\partial_{b_1} X^c) T ^{a_1 \ldots a_r}{}_{c b_2 \ldots b_s} + \ldots + (\partial_{b_s} X^c) T ^{a_1 \ldots a_r}{}_{b_1 \ldots b_{s-1} c} + w (\partial_{c} X^c) T ^{a_1 \ldots a_r}{}_{b_1 \ldots b_{s}}
\end{align}

Thus to evaluate the Lie derivative, one must first know the tensor density weight $w$ for each tensor. In this formulation of Einstein's equations, **all evolved quantities have density weight $w=0$**, so according to the definition of Lie derivative above,
\begin{align}
\mathcal{L}_\beta \bar{\gamma}_{ij} &= \beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik}, \\
\mathcal{L}_\beta \bar{A}_{ij} &= \beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik}, \\
\mathcal{L}_\beta \phi &= \beta^k \partial_k \phi, \\
\mathcal{L}_\beta K &= \beta^k \partial_k K, \\
\mathcal{L}_\beta \bar{\Lambda}^i &= \beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k
\end{align}

With these definitions, the BSSN equations for the un-rescaled evolved variables in the form $\partial_t \vec{U} = f\left(\vec{U},\partial_i \vec{U}, \partial_i \partial_j \vec{U},...\right)$ become

\begin{align}
  \partial_t \bar{\gamma}_{i j} {} = {} & \left[\beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik} \right] + \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_t \bar{A}_{i j} {} = {} & \left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right] - \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_t \phi {} = {} & \left[\beta^k \partial_k \phi \right] + \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{t} K {} = {} & \left[\beta^k \partial_k K \right] + \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_t \bar{\Lambda}^{i} {} = {} & \left[\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k \right] + \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \alpha \bar{A}^{j k} \Delta_{j k}^{i}  -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\partial_t \alpha &= \left[\beta^i \partial_i \alpha\right] - 2 \alpha K \\
  \partial_{t} \beta^{i} &= \left[\beta^j \partial_j \beta^i\right] + B^{i} \\
  \partial_{t} B^{i} &= \left[\beta^j \partial_j B^i\right] + \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i}, \; .
\end{align}

where the terms moved from the right-hand sides to the left-hand sides are enclosed in square braces. 

Notice that the shift advection operator $\beta^k \partial_k \left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\}$ appears on the right-hand side of *every* expression. As the shift determines how the spatial coordinates $x^i$ move on the next 3D slice of our 4D manifold, we find that representing $\partial_k$ in these shift advection terms via an *upwinded* finite difference stencil results in far lower numerical errors. This trick is implemented below in all shift advection terms.

As discussed in the [NRPy+ tutorial module on BSSN quantities](Tutorial-BSSN_quantities.ipynb), tensorial expressions can diverge at coordinate singularities, so each tensor in the set of BSSN variables

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$

is written in terms of the corresponding rescaled quantity in the set

$$\left\{h_{i j},a_{i j},\text{cf}, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\},$$ 

as defined in the [BSSN quantities tutorial](Tutorial-BSSN_quantities.ipynb). 

Note that $\partial_{0} \bar{\Lambda}^{i}$ in the right-hand side of the $\partial_{t} B^{i}$ equation is computed by adding $\beta^j \partial_j \bar{\Lambda}^i$ to the right-hand side expression given for $\partial_t \bar{\Lambda}^i$, so no explicit time dependence occurs in the right-hand sides of the BSSN evolution equations and the Method of Lines can be applied directly.

## Numerical Implementation: Putting it all together.

Recall that according to our tensor rescaling prescription, each time an unrescaled variable appears within a derivative, it is replaced by a rescaled variable multiplied by the rescaling factor ReU\[\] or ReDD\[\]\[\]. Then the derivative expression is replaced using the product rule, where derivatives of ReU or ReDD are computed exactly and the rescaled variable will be computed using numerical approximations (e.g., finite difference derivatives).

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
rfm.reference_metric()
from outputC import *

Next we'll need to initialize parameters and register the set of evolved gridfunctions, which are all rescaled variables $\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}$:

## Right-hand sides of BSSN equations, part 1: $\partial_t \bar{\gamma}_{i j}$

Now that the most difficult quantity to compute is out of the way, we now have a template for moving forward with the rest of the BSSN equations. In the following, we will evaluate the right-hand sides of each non-evolved variable, and then rescale the result to correspond to the evolved variables as our last step. 

Before we get started, one final numerical detail: all shift advection terms (i.e., terms of the form $\beta^i \partial_i$) are upwinded according to the direction of the shift vector. Upwinded derivatives are given by the "_dupD" variable suffix.

Let's start with

$$
\partial_t \bar{\gamma}_{i j} = 
{\underbrace {\textstyle \left[\beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik} \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right )}_{\text{Term 2}}}
{\underbrace {\textstyle -2 \alpha \bar{A}_{i j}}_{\text{Term 3}}}.
$$

### Term 1 of $\partial_t \bar{\gamma}_{i j} =$ gammabar_rhsDD\[i\]\[j\]: $\beta^k \bar{\gamma}_{ij,k} + \beta^k_{,i} \bar{\gamma}_{kj} + \beta^k_{,j} \bar{\gamma}_{ik}$

We first need to define the rescaled version of $\beta^i$, $\mathcal{B}^i$=vetU\[i\] and betaU_dD\[i\]\[j\] in terms of the rescaled derivative.

In [7]:
# Step 8a: Define \beta^i and \beta^i_{,k} in terms of rescaled quantity vetU[i] and vetU_dD[i][j]:
betaU = ixp.zerorank1()
for i in range(DIM):
    betaU[i] = vetU[i]*rfm.ReU[i]

vetU_dD = ixp.declarerank2("vetU_dD","nosym")
vetU_dupD = ixp.declarerank2("vetU_dupD","nosym") # Needed for \beta^i RHS
vetU_dDD = ixp.declarerank3("vetU_dDD","sym12") # Needed for \bar{\Lambda}^i RHS
betaU_dD = ixp.zerorank2()
betaU_dupD = ixp.zerorank2() # Needed for \beta^i RHS
betaU_dDD = ixp.zerorank3() # Needed for \bar{\Lambda}^i RHS
for i in range(DIM):
    for j in range(DIM):
        betaU_dD[i][j] = vetU_dD[i][j]*rfm.ReU[i] + vetU[i]*rfm.ReUdD[i][j]
        betaU_dupD[i][j] = vetU_dupD[i][j]*rfm.ReU[i] + vetU[i]*rfm.ReUdD[i][j] # Needed for \beta^i RHS
        for k in range(DIM):
            # Needed for \bar{\Lambda}^i RHS:
            betaU_dDD[i][j][k] = vetU_dDD[i][j][k]*rfm.ReU[i]  + vetU_dD[i][j]*rfm.ReUdD[i][k] + \
                                 vetU_dD[i][k]*rfm.ReUdD[i][j] + vetU[i]*rfm.ReUdDD[i][j][k]

# Step 8b: First term of \partial_t \bar{\gamma}_{i j} right-hand side:
# \beta^k \bar{\gamma}_{ij,k} + \beta^k_{,i} \bar{\gamma}_{kj} + \beta^k_{,j} \bar{\gamma}_{ik}
gammabar_rhsDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            gammabar_rhsDD[i][j] += betaU[k]*gammabarDD_dupD[i][j][k] + betaU_dD[k][i]*gammabarDD[k][j] \
                                                                      + betaU_dD[k][j]*gammabarDD[i][k]

### Term 2 of $\partial_t \bar{\gamma}_{i j} =$ gammabar_rhsDD\[i\]\[j\]: $\frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right )$

Let's first convert this expression to be in terms of the evolved variables $a_{ij}$ and $\mathcal{B}^i$, starting with $\bar{A}_{ij} = a_{ij} \text{ReDD[i][j]}$. Then $\bar{A}^k_{k} = \bar{\gamma}^{ij} \bar{A}_{ij}$, and we have already defined $\bar{\gamma}^{ij}$ in terms of the evolved quantity $h_{ij}$.

Next, we wish to compute 

$$\bar{D}_{k} \beta^{k} = \beta^k_{,k} + \frac{\beta^k \bar{\gamma}_{,k}}{2 \bar{\gamma}},$$

where $\bar{\gamma}$ is the determinant of the conformal metric $\bar{\gamma}_{ij}$. *Exercise to student: Prove the above relation.*
[Solution.](https://physics.stackexchange.com/questions/81453/general-relativity-christoffel-symbol-identity)

Usually (i.e., so long as we make the parameter choice ${\rm detgbarOverdetghat\_equals\_one=``False''}$) we will choose $\bar{\gamma}=\hat{\gamma}$, so $\bar{\gamma}$ will in general possess coordinate singularities. Thus we would prefer to rewrite derivatives of $\bar{\gamma}$ in terms of derivatives of $\bar{\gamma}/\hat{\gamma} = 1$.

In [8]:
# Step 8c: Define \bar{A}_{ij} = a_{ij} \text{ReDD[i][j]} = AbarDD[i][j], and its contraction trAbar = \bar{A}^k_k
AbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        AbarDD[i][j] = aDD[i][j]*rfm.ReDD[i][j]
trAbar = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        trAbar += gammabarUU[i][j]*AbarDD[i][j]

# Step 8d: Define detgammabar, detgammabar_dD, and detgammabar_dDD (needed for \partial_t \bar{\Lambda}^i below)
detgammabar = detgbarOverdetghat * rfm.detgammahat

detgammabar_dD = ixp.zerorank1()
if par.parval_from_str("BSSN_RHSs::detgbarOverdetghat_equals_one") == "False":
    detgbarOverdetghat_dD = ixp.declarerank1("detgbarOverdetghat_dD")
else:
    detgbarOverdetghat_dD = ixp.zerorank1()

for i in range(DIM):
    detgammabar_dD[i] = detgbarOverdetghat_dD[i]*rfm.detgammahat + detgbarOverdetghat*rfm.detgammahatdD[i]

detgammabar_dDD = ixp.zerorank2()
if par.parval_from_str("BSSN_RHSs::detgbarOverdetghat_equals_one") == "False":
    detgbarOverdetghat_dDD = ixp.declarerank2("detgbarOverdetghat_dDD","sym01")
else:
    detgbarOverdetghat_dDD = ixp.zerorank2()

for i in range(DIM):
    for j in range(DIM):
        detgammabar_dDD[i][j] = detgbarOverdetghat_dDD[i][j]*rfm.detgammahat +   \
                                detgbarOverdetghat_dD[i]*rfm.detgammahatdD[j] + \
                                detgbarOverdetghat_dD[j]*rfm.detgammahatdD[i] + \
                                detgbarOverdetghat*rfm.detgammahatdDD[i][j]

# Step 8e: Compute the contraction \bar{D}_k \beta^k = \beta^k_{,k} + \frac{\beta^k \bar{\gamma}_{,k}}{2 \bar{\gamma}}
Dbarbetacontraction = sp.sympify(0)
for k in range(DIM):
    Dbarbetacontraction += betaU_dD[k][k] + betaU[k]*detgammabar_dD[k]/(2*detgammabar)

# Step 8f: Second term of \partial_t \bar{\gamma}_{i j} right-hand side:
# \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right )
for i in range(DIM):
    for j in range(DIM):
        gammabar_rhsDD[i][j] += sp.Rational(2,3)*gammabarDD[i][j]*(alpha*trAbar - Dbarbetacontraction)

### Term 3 of $\partial_t \bar{\gamma}_{i j} =$ gammabar_rhsDD\[i\]\[j\]: $-2 \alpha \bar{A}_{ij}$

In [9]:
# Step 8g: Third term of \partial_t \bar{\gamma}_{i j} right-hand side:
# -2 \alpha \bar{A}_{ij}
for i in range(DIM):
    for j in range(DIM):
        gammabar_rhsDD[i][j] += -2*alpha*AbarDD[i][j]

## Right-hand sides of BSSN equations, part 2: $\partial_t \bar{A}_{i j}$

$$\partial_t \bar{A}_{i j} = 
{\underbrace {\textstyle \left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right]}_{\text{Term 1}}}
{\underbrace {\textstyle - \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K}_{\text{Term 2}}} + 
{\underbrace {\textstyle e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}}}_{\text{Term 3}}}$$

### Term 1 of $\partial_t \bar{A}_{i j}$ = Abar_rhsDD\[i\]\[j\]: $\left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right]$

Notice the first subexpression has a $\beta^k \partial_k A_{ij}$ advection term, which will be upwinded.

In [10]:
# Step 9a: First term of \partial_t \bar{A}_{i j}:
# \beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik}

# First define aDD_dupD:
AbarDD_dupD = ixp.zerorank3()
aDD_dupD = ixp.declarerank3("aDD_dupD","sym01")
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            AbarDD_dupD[i][j][k] += aDD_dupD[i][j][k]*rfm.ReDD[i][j] + aDD[i][j]*rfm.ReDDdD[i][j][k]

Abar_rhsDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            Abar_rhsDD[i][j] += betaU[k]*AbarDD_dupD[i][j][k] + betaU_dD[k][i]*AbarDD[k][j] \
                                                              + betaU_dD[k][j]*AbarDD[i][k]

### Term 2 of $\partial_t \bar{A}_{i j}$ = Abar_rhsDD\[i\]\[j\]: $- \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} \bar{A}^{k}_{j} + \alpha \bar{A}_{i j} K$

Note that $\bar{D}_{k} \beta^{k}$ was already defined as "Dbarbetacontraction".

In [11]:
# Step 9b: Second term of \partial_t \bar{A}_{i j}:
# - (2/3) \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K
AbarUD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            AbarUD[i][j] += gammabarUU[i][k]*AbarDD[k][j]

for i in range(DIM):
    for j in range(DIM):
        Abar_rhsDD[i][j] += -sp.Rational(2,3)*AbarDD[i][j]*Dbarbetacontraction + alpha*AbarDD[i][j]*trK
        for k in range(DIM):
            Abar_rhsDD[i][j] += -2*alpha * AbarDD[i][k]*AbarUD[k][j]

### Term 3 of $\partial_t \bar{A}_{i j}$ = Abar_rhsDD\[i\]\[j\]: $e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}}$

The first covariant derivatives of $\phi$ and $\alpha$ are simply partial derivatives. However, $\phi$ is not a gridfunction; "cf" is. cf="W" (default value) denotes that the evolved variable is $W=e^{-2 \phi}$, which results in smoother spacetime fields around puncture black holes (desirable).

In [12]:
# Step 9c: Define partial derivatives of \phi in terms of evolved quantity "cf":
cf_dD = ixp.declarerank1("cf_dD")
cf_dupD = ixp.declarerank1("cf_dupD") # Needed for \partial_t \phi next.
cf_dDD = ixp.declarerank2("cf_dDD","sym01")
phi_dD = ixp.zerorank1()
phi_dupD = ixp.zerorank1()
phi_dDD = ixp.zerorank2()
exp_m4phi = sp.sympify(0)
if par.parval_from_str("BSSN_RHSs::ConformalFactor") == "phi":
    for i in range(DIM):
        phi_dD[i] = cf_dD[i]
        phi_dupD[i] = cf_dupD[i]
        for j in range(DIM):
            phi_dDD[i][j] = cf_dDD[i][j]
    exp_m4phi = sp.exp(-4*cf)
elif par.parval_from_str("BSSN_RHSs::ConformalFactor") == "W":
    # \partial_i W = \partial_i (e^{-2 phi}) = -2 e^{-2 phi} \partial_i phi
    # -> \partial_i phi = -\partial_i cf / (2 cf)
    for i in range(DIM):
        phi_dD[i] = - cf_dD[i] / (2*cf)
        phi_dupD[i] = - cf_dupD[i] / (2*cf)
        for j in range(DIM):
            # \partial_j \partial_i phi = - \partial_j [\partial_i cf / (2 cf)]
            #                           = - cf_{,ij} / (2 cf) + \partial_i cf \partial_j cf / (2 cf^2)
            phi_dDD[i][j] = (- cf_dDD[i][j] + cf_dD[i]*cf_dD[j] / cf) / (2*cf)
    exp_m4phi = cf*cf
elif par.parval_from_str("BSSN_RHSs::ConformalFactor") == "chi":
    # \partial_i chi = \partial_i (e^{-4 phi}) = -4 e^{-4 phi} \partial_i phi
    # -> \partial_i phi = -\partial_i cf / (4 cf)
    for i in range(DIM):
        phi_dD[i] = - cf_dD[i] / (4*cf)
        phi_dupD[i] = - cf_dupD[i] / (4*cf)
        for j in range(DIM):
            # \partial_j \partial_i phi = - \partial_j [\partial_i cf / (4 cf)]
            #                           = - cf_{,ij} / (4 cf) + \partial_i cf \partial_j cf / (4 cf^2)
            phi_dDD[i][j] = (- cf_dDD[i][j] + cf_dD[i]*cf_dD[j] / cf) / (4*cf)
    exp_m4phi = cf
else:
    print("Error: ConformalFactor == "+par.parval_from_str("BSSN_RHSs::ConformalFactor")+" unsupported!")
    exit(1)
    
# Step 9d: Define phi_dBarD = phi_dD (since phi is a scalar) and phi_dBarDD (covariant derivative)
#          \bar{D}_i \bar{D}_j \phi = \phi_{;\bar{i}\bar{j}} = \bar{D}_i \phi_{,j}
#                                   = \phi_{,ij} - \bar{\Gamma}^k_{ij} \phi_{,k}
phi_dBarD = phi_dD
phi_dBarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        phi_dBarDD[i][j] = phi_dDD[i][j]
        for k in range(DIM):
            phi_dBarDD[i][j] += - GammabarUDD[k][i][j]*phi_dD[k]

# Step 9e: Define first and second derivatives of \alpha, as well as 
#         \bar{D}_i \bar{D}_j \alpha, which is defined just like phi
alpha_dD = ixp.declarerank1("alpha_dD")
alpha_dDD = ixp.declarerank2("alpha_dDD","sym01")
alpha_dBarD = alpha_dD
alpha_dBarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        alpha_dBarDD[i][j] = alpha_dDD[i][j]
        for k in range(DIM):
            alpha_dBarDD[i][j] += - GammabarUDD[k][i][j]*alpha_dD[k]

# Step 9f: Define the terms in curly braces:
curlybrackettermsDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        curlybrackettermsDD[i][j] = -2*alpha*phi_dBarDD[i][j] + 4*alpha*phi_dBarD[i]*phi_dBarD[j] \
                                    +2*alpha_dBarD[i]*phi_dBarD[j] \
                                    +2*alpha_dBarD[j]*phi_dBarD[i] \
                                    -alpha_dBarDD[i][j] + alpha*RbarDD[i][j]

# Step 9g: Compute the trace:
curlybracketterms_trace = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        curlybracketterms_trace += gammabarUU[i][j]*curlybrackettermsDD[i][j]

# Step 9h: Third and final term of Abar_rhsDD[i][j]:
for i in range(DIM):
    for j in range(DIM):
        Abar_rhsDD[i][j] += exp_m4phi*(curlybrackettermsDD[i][j] - \
                                       sp.Rational(1,3)*gammabarDD[i][j]*curlybracketterms_trace)   

## Right-hand sides of BSSN equations, part 3: $\partial_t \phi \to \partial_t$ cf

$$\partial_t \phi = 
{\underbrace {\textstyle \left[\beta^k \partial_k \phi \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right)}_{\text{Term 2}}}$$

The right-hand side of $\partial_t \phi$ is trivial except for the fact that the actual evolved variable is "cf" (short for conformal factor), which could represent
* cf = $\phi$
* cf = $W = e^{-2 \phi}$ (default)
* cf = $\chi = e^{-4 \phi}$

Thus we are actually computing the right-hand side of the equation $\partial_t $cf, which is related to $\partial_t \phi$ via simple relations:
* cf = $\phi$: $\partial_t $cf$ = \partial_t \phi$ (unchanged)
* cf = $W$: $\partial_t $cf$ = \partial_t (e^{-2 \phi}) = -2 e^{-2\phi}\partial_t \phi = -2 W \partial_t \phi$. Thus we need to multiply the right-hand side by $-2 W = -2$cf when cf = $W$.
* cf = $\chi$: Same argument as for $W$, except the right-hand side must be multiplied by $-4 \chi=-4$cf.

In [13]:
# Step 10: right-hand side of conformal factor variable "cf". Supported
#          options include: cf=phi, cf=W=e^(-2*phi) (default), and cf=chi=e^(-4*phi)
# \partial_t phi = \left[\beta^k \partial_k \phi \right] <- TERM 1
#                  + \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) <- TERM 2

cf_rhs = sp.Rational(1,6) * (Dbarbetacontraction - alpha*trK) # Term 2
for k in range(DIM):
    cf_rhs += betaU[k]*phi_dupD[k] # Term 1

# Next multiply to convert phi_rhs to cf_rhs.
if par.parval_from_str("BSSN_RHSs::ConformalFactor") == "phi":
    pass # do nothing; cf_rhs = phi_rhs
elif par.parval_from_str("BSSN_RHSs::ConformalFactor") == "W":
    cf_rhs *= -2*cf # cf_rhs = -2*cf*phi_rhs
elif par.parval_from_str("BSSN_RHSs::ConformalFactor") == "chi":
    cf_rhs *= -4*cf # cf_rhs = -4*cf*phi_rhs
else:
    print("Error: ConformalFactor == "+par.parval_from_str("BSSN_RHSs::ConformalFactor")+" unsupported!")
    exit(1) 

## Right-hand sides of BSSN equations, part 4: $\partial_t K$

$$
\partial_{t} K = 
{\underbrace {\textstyle \left[\beta^k \partial_k K \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \frac{1}{3} \alpha K^{2}}_{\text{Term 2}}} +
{\underbrace {\textstyle \alpha \bar{A}_{i j} \bar{A}^{i j}}_{\text{Term 3}}}
{\underbrace {\textstyle - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right )}_{\text{Term 4}}}
$$

In [14]:
# Step 11: right-hand side of trK (trace of extrinsic curvature):
# \partial_t K = \beta^k \partial_k K <- TERM 1
#           + \frac{1}{3} \alpha K^{2} <- TERM 2
#           + \alpha \bar{A}_{i j} \bar{A}^{i j} <- TERM 3
#           - - e^{-4 \phi} (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi ) <- TERM 4
trK_rhs = sp.Rational(1,3)*alpha*trK*trK # Term 2
trK_dupD = ixp.declarerank1("trK_dupD")
for k in range(DIM):
    trK_rhs += betaU[k]*trK_dupD[k] # Term 1
for i in range(DIM):
    for j in range(DIM):
        trK_rhs += -exp_m4phi*gammabarUU[i][j]*(alpha_dBarDD[i][j] + 2*alpha_dBarD[j]*phi_dBarD[i]) # Term 4
AbarUU = ixp.zerorank2() # Needed also for \partial_t \bar{\Lambda}^i
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                AbarUU[i][j] += gammabarUU[i][k]*gammabarUU[j][l]*AbarDD[k][l]
for i in range(DIM):
    for j in range(DIM):
        trK_rhs += alpha*AbarDD[i][j]*AbarUU[i][j] # Term 3

## Right-hand sides of BSSN equations, part 5: $\partial_t \bar{\Lambda}^{i}$

\begin{align}
\partial_t \bar{\Lambda}^{i} &= 
{\underbrace {\textstyle \left[\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i}}_{\text{Term 2}}} + 
{\underbrace {\textstyle \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j}}_{\text{Term 3}}} + 
{\underbrace {\textstyle \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j}}_{\text{Term 4}}} \nonumber \\
  & 
{\underbrace {\textstyle - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \alpha \partial_{j} \phi \right )}_{\text{Term 5}}} + 
{\underbrace {\textstyle 2 \alpha \bar{A}^{j k} \Delta_{j k}^{i}}_{\text{Term 6}}} 
{\underbrace {\textstyle -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K}_{\text{Term 7}}}
\end{align}

### Term 1 of  $\partial_t \bar{\Lambda}^{i}$: $\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k$

Computing this term requires that we define $\bar{\Lambda}^i$ and $\bar{\Lambda}^i_{,j}$ in terms of the rescaled (i.e., actual evolved) variable $\lambda^i$ and derivatives: 
\begin{align}
\bar{\Lambda}^i &= \lambda^i \text{ReU[i]} \\
\bar{\Lambda}^i_{,\ j} &= \lambda^i_{,\ j} \text{ReU[i]} + \lambda^i \text{ReUdD[i][j]} 
\end{align}

In [15]:
# Step 12: right-hand side of \partial_t \bar{\Lambda}^i:
# \partial_t \bar{\Lambda}^i = \beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k <- TERM 1
#                            + \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} <- TERM 2
#                            + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} <- TERM 3
#                            + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} <- TERM 4
#                            - 2 \bar{A}^{i j} (\partial_{j} \alpha - 6 \partial_{j} \phi) <- TERM 5
#                            + 2 \alpha \bar{A}^{j k} \Delta_{j k}^{i} <- TERM 6
#                            - \frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K <- TERM 7

# Step 12a: Term 1 of \partial_t \bar{\Lambda}^i: \beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k
# First we declare \bar{\Lambda}^i and \bar{\Lambda}^i_{,j} in terms of \lambda^i and \lambda^i_{,j}
LambarU = ixp.zerorank1()
LambarU_dupD = ixp.zerorank2()
lambdaU_dupD = ixp.declarerank2("lambdaU_dupD","nosym")
for i in range(DIM):
    LambarU[i] = lambdaU[i]*rfm.ReU[i]
    for j in range(DIM):
        LambarU_dupD[i][j] = lambdaU_dupD[i][j]*rfm.ReU[i] + lambdaU[i]*rfm.ReUdD[i][j]

Lambar_rhsU = ixp.zerorank1()
for i in range(DIM):
    for k in range(DIM):
        Lambar_rhsU[i] += betaU[k]*LambarU_dupD[i][k] - betaU_dD[i][k]*LambarU[k] # Term 1

### Term 2 of $\partial_t \bar{\Lambda}^{i}$: $\bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i}$

This is a relatively difficult term to compute, as it requires we evaluate the second covariant derivative of the shift vector, with respect to the hatted (i.e., reference) metric.

Based on the definition of covariant derivative, we have
$$
\hat{D}_{k} \beta^{i} = \beta^i_{,k} + \hat{\Gamma}^i_{mk} \beta^m
$$

Since $\hat{D}_{k} \beta^{i}$ is a tensor, the covariant derivative of this will have the same indexing as a tensor $T_k^i$:

$$
\hat{D}_{j} T^i_k = T^i_{k,j} + \hat{\Gamma}^i_{dj} T^d_k - \hat{\Gamma}^d_{kj} T^i_d.
$$

Therefore,
\begin{align}
\hat{D}_{j} \left(\hat{D}_{k} \beta^{i}\right) &= \left(\beta^i_{,k} + \hat{\Gamma}^i_{mk} \beta^m\right)_{,j} + \hat{\Gamma}^i_{dj} \left(\beta^d_{,k} + \hat{\Gamma}^d_{mk} \beta^m\right) - \hat{\Gamma}^d_{kj} \left(\beta^i_{,d} + \hat{\Gamma}^i_{md} \beta^m\right) \\
&= \beta^i_{,kj} + \hat{\Gamma}^i_{mk,j} \beta^m + \hat{\Gamma}^i_{mk} \beta^m_{,j} + \hat{\Gamma}^i_{dj}\beta^d_{,k} + \hat{\Gamma}^i_{dj}\hat{\Gamma}^d_{mk} \beta^m - \hat{\Gamma}^d_{kj} \beta^i_{,d} - \hat{\Gamma}^d_{kj} \hat{\Gamma}^i_{md} \beta^m \\
&= {\underbrace {\textstyle \beta^i_{,kj}}_{\text{Term 2a}}}
{\underbrace {\textstyle \hat{\Gamma}^i_{mk,j} \beta^m + \hat{\Gamma}^i_{mk} \beta^m_{,j} + \hat{\Gamma}^i_{dj}\beta^d_{,k} - \hat{\Gamma}^d_{kj} \beta^i_{,d}}_{\text{Term 2b}}} +
{\underbrace {\textstyle \hat{\Gamma}^i_{dj}\hat{\Gamma}^d_{mk} \beta^m - \hat{\Gamma}^d_{kj} \hat{\Gamma}^i_{md} \beta^m}_{\text{Term 2c}}},
\end{align}

where 
$$
\text{Term 2} = \bar{\gamma}^{jk} \left(\text{Term 2a} + \text{Term 2b} + \text{Term 2c}\right)
$$

In [16]:
# Step 12b: Term 2 of \partial_t \bar{\Lambda}^i = \bar{\gamma}^{jk} (Term 2a + Term 2b + Term 2c)
# Term 2a: \bar{\gamma}^{jk} \beta^i_{,kj}
Term2aUDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            Term2aUDD[i][j][k] += betaU_dDD[i][k][j]
# Term 2b: \hat{\Gamma}^i_{mk,j} \beta^m + \hat{\Gamma}^i_{mk} \beta^m_{,j}
#          + \hat{\Gamma}^i_{dj}\beta^d_{,k} - \hat{\Gamma}^d_{kj} \beta^i_{,d}
Term2bUDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for m in range(DIM):
                Term2bUDD[i][j][k] += rfm.GammahatUDDdD[i][m][k][j]*betaU[m]    \
                                      + rfm.GammahatUDD[i][m][k]*betaU_dD[m][j] \
                                      + rfm.GammahatUDD[i][m][j]*betaU_dD[m][k] \
                                      - rfm.GammahatUDD[m][k][j]*betaU_dD[i][m]
# Term 2c: \hat{\Gamma}^i_{dj}\hat{\Gamma}^d_{mk} \beta^m - \hat{\Gamma}^d_{kj} \hat{\Gamma}^i_{md} \beta^m
Term2cUDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for m in range(DIM):
                for d in range(DIM):
                    Term2cUDD[i][j][k] += ( rfm.GammahatUDD[i][d][j]*rfm.GammahatUDD[d][m][k] \
                                           -rfm.GammahatUDD[d][k][j]*rfm.GammahatUDD[i][m][d])*betaU[m]

Lambar_rhsUpieceU = ixp.zerorank1()

# Put it all together to get Term 2:
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            Lambar_rhsU[i] += gammabarUU[j][k] * (Term2aUDD[i][j][k] + Term2bUDD[i][j][k] + Term2cUDD[i][j][k])
            Lambar_rhsUpieceU[i] += gammabarUU[j][k] * (Term2aUDD[i][j][k] + Term2bUDD[i][j][k] + Term2cUDD[i][j][k])

### Term 3 of  $\partial_t \bar{\Lambda}^{i}$: $\frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j}$

This term is the simplest to implement, as $\bar{D}_{j} \beta^{j}$ and $\Delta^i$ have already been defined, as Dbarbetacontraction and DGammaU\[i\], respectively:

In [17]:
# Step 12c: Term 3 of \partial_t \bar{\Lambda}^i:
#    \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j}
for i in range(DIM):
    Lambar_rhsU[i] += sp.Rational(2,3)*DGammaU[i]*Dbarbetacontraction # Term 3

### Term 4 of  $\partial_t \bar{\Lambda}^{i}$: $\frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j}$

Recall first that

$$\bar{D}_{k} \beta^{k} = \beta^k_{,\ k} + \frac{\beta^k \bar{\gamma}_{,k}}{2 \bar{\gamma}},$$
which is a scalar, so

\begin{align}
\bar{D}_m \bar{D}_{j} \beta^{j} &= \left(\beta^k_{,\ k} + \frac{\beta^k \bar{\gamma}_{,k}}{2 \bar{\gamma}}\right)_{,m} \\
&= \beta^k_{\ ,km} + \frac{\beta^k_{\ ,m} \bar{\gamma}_{,k} + \beta^k \bar{\gamma}_{\ ,km}}{2 \bar{\gamma}} - \frac{\beta^k \bar{\gamma}_{,k} \bar{\gamma}_{,m}}{2 \bar{\gamma}^2}
\end{align}

Thus, 
\begin{align}
\bar{D}^i \bar{D}_{j} \beta^{j} 
&= \bar{\gamma}^{im} \bar{D}_m \bar{D}_{j} \beta^{j} \\
&= \bar{\gamma}^{im} \left(\beta^k_{\ ,km} + \frac{\beta^k_{\ ,m} \bar{\gamma}_{,k} + \beta^k \bar{\gamma}_{\ ,km}}{2 \bar{\gamma}} - \frac{\beta^k \bar{\gamma}_{,k} \bar{\gamma}_{,m}}{2 \bar{\gamma}^2} \right)
\end{align}

In [18]:
# Step 12d: Term 4 of \partial_t \bar{\Lambda}^i:
#           \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j}
Dbarbetacontraction_dBarD = ixp.zerorank1()
for k in range(DIM):
    for m in range(DIM):
        Dbarbetacontraction_dBarD[m] += betaU_dDD[k][k][m] + \
                                        (betaU_dD[k][m]*detgammabar_dD[k] + 
                                         betaU[k]*detgammabar_dDD[k][m])/(2*detgammabar) \
                                        -betaU[k]*detgammabar_dD[k]*detgammabar_dD[m]/(2*detgammabar*detgammabar)
for i in range(DIM):
    for m in range(DIM):
        Lambar_rhsU[i] += sp.Rational(1,3)*gammabarUU[i][m]*Dbarbetacontraction_dBarD[m]

### Term 5 of  $\partial_t \bar{\Lambda}^{i}$: $- 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6\alpha \partial_{j} \phi\right)$

In [19]:
# Step 12e: Term 5 of \partial_t \bar{\Lambda}^i:
#           - 2 \bar{A}^{i j} (\partial_{j} \alpha - 6 \alpha \partial_{j} \phi)
for i in range(DIM):
    for j in range(DIM):
        Lambar_rhsU[i] += -2*AbarUU[i][j]*(alpha_dD[j] - 6*alpha*phi_dD[j])

### Term 6 of $\partial_t \bar{\Lambda}^{i}$: $2\alpha \bar{A}^{j k} \Delta_{j k}^{i}$

In [20]:
# Step 12f: Term 6 of \partial_t \bar{\Lambda}^i:
#           2 \alpha \bar{A}^{j k} \Delta^{i}_{j k}
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            Lambar_rhsU[i] += 2*alpha*AbarUU[j][k]*DGammaUDD[i][j][k]

### Term 7 of $\partial_t \bar{\Lambda}^{i}$: $-\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K$

In [21]:
# Step 12g: Term 7 of \partial_t \bar{\Lambda}^i:
#           -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K
trK_dD = ixp.declarerank1("trK_dD")
for i in range(DIM):
    for j in range(DIM):
        Lambar_rhsU[i] += -sp.Rational(4,3)*alpha*gammabarUU[i][j]*trK_dD[j]

## Right-hand sides of BSSN equations, part 6: $\partial_t \alpha$

After battling through $\partial_t \bar{\Lambda}^i$, we face a straightforward right-hand side:
$$\partial_t \alpha = \left[\beta^i \partial_i \alpha\right] - 2 \alpha K$$

In [22]:
# Step 13: \partial_t \alpha = \beta^i \alpha_{,i} - 2*\alpha*K
alpha_rhs = -2*alpha*trK
alpha_dupD = ixp.declarerank1("alpha_dupD")
for i in range(DIM):
    alpha_rhs += betaU[i]*alpha_dupD[i]

## Right-hand sides of BSSN equations, part 7: $\partial_t \beta^i$

**Option 1: Non-Covariant, Second-Order Shift**
$$\partial_t \beta^i = \left[\beta^j \partial_j \beta^i\right] + B^{i}$$

In [23]:
# Step 14: Set \partial_t \beta^i
beta_rhsU = ixp.zerorank1()
if par.parval_from_str("BSSN_RHSs::ShiftEvolutionOption") == "GammaDriving2ndOrder_NoCovariant":
    # Step 14 Option 1: \partial_t \beta^i = \beta^j \beta^i_{,j} + B^i
    # First define BU, in terms of rescaled variable \bet^i
    BU = ixp.zerorank1()
    for i in range(DIM):
        BU[i] = betU[i]*rfm.ReU[i]

    # Then compute right-hand side:
    for i in range(DIM):
        beta_rhsU[i] += BU[i]
        for j in range(DIM):
            beta_rhsU[i] += betaU[j]*betaU_dupD[i][j]

**Option 2: Covariant, Second-Order Shift**

This is [Brown's](https://arxiv.org/abs/0902.3652) suggested formulation (Eq. 20b; note that Eq. 20a is the same as our lapse condition, as $\bar{D}_j \alpha = \partial_j \alpha$ for scalar $\alpha$):
$$\partial_t \beta^i = \left[\beta^j \bar{D}_j \beta^i\right] + B^{i}$$
Based on the definition of covariant derivative, we have
$$
\bar{D}_{j} \beta^{i} = \beta^i_{,j} + \bar{\Gamma}^i_{mj} \beta^m,
$$
so the above equation becomes
\begin{align}
\partial_t \beta^i &= \left[\beta^j \left(\beta^i_{,j} + \bar{\Gamma}^i_{mj} \beta^m\right)\right] + B^{i}\\
&= {\underbrace {\textstyle \beta^j \beta^i_{,j}}_{\text{Term 1}}} + 
{\underbrace {\textstyle \beta^j \bar{\Gamma}^i_{mj} \beta^m}_{\text{Term 2}}} + 
{\underbrace {\textstyle B^i}_{\text{Term 3}}} 
\end{align}


In [24]:
if par.parval_from_str("BSSN_RHSs::ShiftEvolutionOption") == "GammaDriving2ndOrder_Covariant":
    # Step 14 Option 2: \partial_t \beta^i = \left[\beta^j \bar{D}_j \beta^i\right] + B^{i}
    # First define BU, in terms of rescaled variable \bet^i
    BU = ixp.zerorank1()
    for i in range(DIM):
        BU[i] = betU[i]*rfm.ReU[i]

    # Then compute right-hand side:
    # Term 1: \beta^j \beta^i_{,j}
    for i in range(DIM):
        for j in range(DIM):
            beta_rhsU[i] += betaU[j]*betaU_dupD[i][j]

    # Term 2: \beta^j \bar{\Gamma}^i_{mj} \beta^m
    for i in range(DIM):
        for j in range(DIM):
            for m in range(DIM):
                beta_rhsU[i] += betaU[j]*GammabarUDD[i][m][j]*betaU[m]
    # Term 3: B^i
    for i in range(DIM):
        beta_rhsU[i] += BU[i]

## Right-hand sides of BSSN equations, part 8: $\partial_t B^i$

**Option 1: Non-Covariant, Second-Order Shift**
$$\partial_t B^i = \left[\beta^j \partial_j B^i\right] + \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i}$$

In [25]:
# Step 15: Evaluate \partial_t B^i

# Step 15a: Declare free parameter eta and B_rhsU
eta = par.Cparameters("REAL",thismodule,["eta"])
B_rhsU = ixp.zerorank1()

if par.parval_from_str("BSSN_RHSs::ShiftEvolutionOption") == "GammaDriving2ndOrder_NoCovariant":
    # Step 15: Non-covariant option:
    #  \partial_t B^i = \beta^j \partial_j B^i
    #                 + \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i}
    # Step 15b: Define BU_dupD, in terms of derivative of rescaled variable \bet^i
    BU_dupD = ixp.zerorank2()
    betU_dupD = ixp.declarerank2("betU_dupD","nosym")
    for i in range(DIM):
        for j in range(DIM):
            BU_dupD[i][j] = betU_dupD[i][j]*rfm.ReU[i] + betU[i]*rfm.ReUdD[i][j]

    # Step 15c: Compute \partial_0 \bar{\Lambda}^i = (\partial_t - \beta^i \partial_i) \bar{\Lambda}^j 
    Lambar_partial0 = ixp.zerorank1()
    for i in range(DIM):
        Lambar_partial0[i] = Lambar_rhsU[i]
    for i in range(DIM):
        for j in range(DIM):
            Lambar_partial0[j] += -betaU[i]*LambarU_dupD[j][i]

    # Step 15d: Evaluate RHS of B^i:
    for i in range(DIM):
        B_rhsU[i] += sp.Rational(3,4)*Lambar_partial0[i] - eta*BU[i]
        for j in range(DIM):
            B_rhsU[i] += betaU[j]*BU_dupD[i][j]

**Option 2: Covariant, Second-Order Shift**

This is [Brown's](https://arxiv.org/abs/0902.3652) suggested formulation (Eq. 20c):
$$\partial_t B^i = \left[\beta^j \bar{D}_j B^i\right] + \frac{3}{4}\left( \partial_t \bar{\Lambda}^{i} - \beta^j \bar{D}_j \bar{\Lambda}^{i} \right) - \eta B^{i}$$

Based on the definition of covariant derivative, we have for vector $V^i$
$$
\bar{D}_{j} V^{i} = V^i_{,j} + \bar{\Gamma}^i_{mj} V^m,
$$
so the above equation becomes
\begin{align}
\partial_t B^i &= \left[\beta^j \left(B^i_{,j} + \bar{\Gamma}^i_{mj} B^m\right)\right] + \frac{3}{4}\left[ \partial_t \bar{\Lambda}^{i} - \beta^j \left(\bar{\Lambda}^i_{,j} + \bar{\Gamma}^i_{mj} \bar{\Lambda}^m\right) \right] - \eta B^{i} \\
&= {\underbrace {\textstyle \beta^j B^i_{,j}}_{\text{Term 1}}} + 
{\underbrace {\textstyle \beta^j \bar{\Gamma}^i_{mj} B^m}_{\text{Term 2}}} + 
{\underbrace {\textstyle \frac{3}{4}\partial_t \bar{\Lambda}^{i}}_{\text{Term 3}}} -
{\underbrace {\textstyle \frac{3}{4}\beta^j \bar{\Lambda}^i_{,j}}_{\text{Term 4}}} -
{\underbrace {\textstyle \frac{3}{4}\beta^j \bar{\Gamma}^i_{mj} \bar{\Lambda}^m}_{\text{Term 5}}} -
{\underbrace {\textstyle \eta B^i}_{\text{Term 6}}}
\end{align}

In [26]:
if par.parval_from_str("BSSN_RHSs::ShiftEvolutionOption") == "GammaDriving2ndOrder_Covariant":
    # Step 15: Covariant option:
    #  \partial_t B^i = \beta^j \bar{D}_j B^i
    #               + \frac{3}{4} ( \partial_t \bar{\Lambda}^{i} - \beta^j \bar{D}_j \bar{\Lambda}^{i} ) 
    #               - \eta B^{i}
    #                 = \beta^j B^i_{,j} + \beta^j \bar{\Gamma}^i_{mj} B^m
    #               + \frac{3}{4}[ \partial_t \bar{\Lambda}^{i} 
    #                            - \beta^j (\bar{\Lambda}^i_{,j} + \bar{\Gamma}^i_{mj} \bar{\Lambda}^m)] 
    #               - \eta B^{i}
    # Term 1, part a: First compute B^i_{,j} using upwinded derivative
    BU_dupD = ixp.zerorank2()
    betU_dupD = ixp.declarerank2("betU_dupD","nosym")
    for i in range(DIM):
        for j in range(DIM):
            BU_dupD[i][j] = betU_dupD[i][j]*rfm.ReU[i] + betU[i]*rfm.ReUdD[i][j]
    # Term 1: \beta^j B^i_{,j}
    for i in range(DIM):
        for j in range(DIM):
            B_rhsU[i] += betaU[j]*BU_dupD[i][j]
    # Term 2: \beta^j \bar{\Gamma}^i_{mj} B^m
    for i in range(DIM):
        for j in range(DIM):
            for m in range(DIM):
                B_rhsU[i] += betaU[j]*GammabarUDD[i][m][j]*BU[m]
    # Term 3: \frac{3}{4}\partial_t \bar{\Lambda}^{i}
    for i in range(DIM):
        B_rhsU[i] += sp.Rational(3,4)*Lambar_rhsU[i]
    # Term 4: -\frac{3}{4}\beta^j \bar{\Lambda}^i_{,j}
    for i in range(DIM):
        for j in range(DIM):
            B_rhsU[i] += -sp.Rational(3,4)*betaU[j]*LambarU_dupD[i][j]
    # Term 5: -\frac{3}{4}\beta^j \bar{\Gamma}^i_{mj} \bar{\Lambda}^m
    for i in range(DIM):
        for j in range(DIM):
            for m in range(DIM):
                B_rhsU[i] += -sp.Rational(3,4)*betaU[j]*GammabarUDD[i][m][j]*LambarU[m]
    # Term 6: - \eta B^i
    for i in range(DIM):
        B_rhsU[i] += -eta*BU[i]

## Rescale right-hand sides of BSSN equations

Next we rescale the right-hand sides of the BSSN equations so that the evolved variables are $\left\{h_{i j},a_{i j},\text{cf}, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}$

In [27]:
# Step 16: Rescale the RHS quantities so that the evolved 
#          variables are smooth across coord singularities
h_rhsDD     = ixp.zerorank2()
a_rhsDD     = ixp.zerorank2()
lambda_rhsU = ixp.zerorank1()
vet_rhsU    = ixp.zerorank1()
bet_rhsU    = ixp.zerorank1()
for i in range(DIM):
    lambda_rhsU[i] = Lambar_rhsU[i] / rfm.ReU[i]
    vet_rhsU[i]    =   beta_rhsU[i] / rfm.ReU[i]
    bet_rhsU[i]    =      B_rhsU[i] / rfm.ReU[i]
    for j in range(DIM):
        h_rhsDD[i][j] = gammabar_rhsDD[i][j] / rfm.ReDD[i][j]
        a_rhsDD[i][j] =     Abar_rhsDD[i][j] / rfm.ReDD[i][j]
#print(str(Abar_rhsDD[2][2]).replace("**","^").replace("_","").replace("xx","x").replace("sin(x2)","Sin[x2]").replace("sin(2*x2)","Sin[2*x2]").replace("cos(x2)","Cos[x2]").replace("detgbaroverdetghat","detg"))
#print(str(Dbarbetacontraction).replace("**","^").replace("_","").replace("xx","x").replace("sin(x2)","Sin[x2]").replace("detgbaroverdetghat","detg"))
#print(betaU_dD)
#print(str(trK_rhs).replace("xx2","xx3").replace("xx1","xx2").replace("xx0","xx1").replace("**","^").replace("_","").replace("sin(xx2)","Sinx2").replace("xx","x").replace("sin(2*x2)","Sin2x2").replace("cos(x2)","Cosx2").replace("detgbaroverdetghat","detg"))
#print(str(bet_rhsU[0]).replace("xx2","xx3").replace("xx1","xx2").replace("xx0","xx1").replace("**","^").replace("_","").replace("sin(xx2)","Sinx2").replace("xx","x").replace("sin(2*x2)","Sin2x2").replace("cos(x2)","Cosx2").replace("detgbaroverdetghat","detg"))

### NRPy+ Module Code Validation

Here, as a code validation check, we verify agreement in the SymPy expressions for the RHSs of the BSSN equations between
1. this tutorial and 
2. the NRPy+ BSSN module.

By default, we analyze the RHSs in Spherical coordinates, though other coordinate systems may be chosen.

In [28]:
# Step 17: We already have SymPy expressions for BSSN RHS expressions
#         in terms of other SymPy variables. Even if we reset the 
#         list of NRPy+ gridfunctions, these *SymPy* expressions for
#         BSSN RHS variables *will remain unaffected*. 
# 
#         Here, we will use the above-defined BSSN RHS expressions
#         to validate against the same expressions in the 
#         BSSN/BSSN_RHSs.py file, to ensure consistency between 
#         this tutorial and the module itself.
#
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []


# Step 18: Call the BSSN_RHSs() function from within the
#          BSSN/BSSN_RHSs.py module,
#          which should do exactly the same as in Steps 1-16 above.
print("vvv Ignore the minor warning below. vvv")

import BSSN.BSSN_RHSs as bssnrhs
bssnrhs.BSSN_RHSs()

print("^^^ Ignore the minor warning above. ^^^\n")

print("Consistency check between BSSN_RHSs tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("alpha_rhs - bssnrhs.alpha_rhs = " + str(alpha_rhs - bssnrhs.alpha_rhs))
print("trK_rhs - bssnrhs.trK_rhs = " + str(trK_rhs - bssnrhs.trK_rhs))
print("cf_rhs - bssnrhs.cf_rhs = " + str(cf_rhs - bssnrhs.cf_rhs))

for i in range(DIM):
    print("vet_rhsU["+str(i)+"] - bssnrhs.vet_rhsU["+str(i)+"] = " + str(vet_rhsU[i] - bssnrhs.vet_rhsU[i]))
    print("bet_rhsU["+str(i)+"] - bssnrhs.bet_rhsU["+str(i)+"] = " + str(bet_rhsU[i] - bssnrhs.bet_rhsU[i]))
    print("lambda_rhsU["+str(i)+"] - bssnrhs.lambda_rhsU["+str(i)+"] = " + 
          str(lambda_rhsU[i] - bssnrhs.lambda_rhsU[i]))
    for j in range(DIM):
        print("h_rhsDD["+str(i)+"]["+str(j)+"] - bssnrhs.h_rhsDD["+str(i)+"]["+str(j)+"] = " 
              + str(h_rhsDD[i][j] - bssnrhs.h_rhsDD[i][j]))
        print("a_rhsDD["+str(i)+"]["+str(j)+"] - bssnrhs.a_rhsDD["+str(i)+"]["+str(j)+"] = " 
              + str(a_rhsDD[i][j] - bssnrhs.a_rhsDD[i][j]))
                

vvv Ignore the minor warning below. vvv
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX
^^^ Ignore the minor warning above. ^^^

Consistency check between BSSN_RHSs tutorial and NRPy+ module: ALL SHOULD BE ZERO.
alpha_rhs - bssnrhs.alpha_rhs = 0
trK_rhs - bssnrhs.trK_rhs = 0
cf_rhs - bssnrhs.cf_rhs = 0
vet_rhsU[0] - bssnrhs.vet_rhsU[0] = 0
bet_rhsU[0] - bssnrhs.bet_rhsU[0] = 0
lambda_rhsU[0] - bssnrhs.lambda_rhsU[0] = 0
h_rhsDD[0][0] - bssnrhs.h_rhsDD[0][0] = 0
a_rhsDD[0][0] - bssnrhs.a_rhsDD[0][0] = 0
h_rhsDD[0][1] - bssnrhs.h_rhsDD[0][1] = 0
a_rhsDD[0][1] - bssnrhs.a_rhsDD[0][1] = 0
h_rhsDD[0][2] - bssnrhs.h_rhsDD[0][2] = 0
a_rhsDD[0][2] - bssnrhs.a_rhsDD[0][2] = 0
vet_rhsU[1] - bssnrhs.vet_rhsU[1] = 0
bet_rhsU[1] - bssnrhs.bet_rhsU[1] = 0
lambda_rhsU[1] - bssnrhs.lambda_rhsU[1] = 0
h_rhsDD[1][0] - bssnrhs.h_rhsDD[1][0] = 0
a_rhsDD[1]

<a id='latex_pdf_output'></a>

# Step 13: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [1]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-BSSN_time_evolution-BSSN_RHSs.ipynb
!pdflatex -interaction=batchmode Tutorial-BSSN_time_evolution-BSSN_RHSs.tex
!pdflatex -interaction=batchmode Tutorial-BSSN_time_evolution-BSSN_RHSs.tex
!pdflatex -interaction=batchmode Tutorial-BSSN_time_evolution-BSSN_RHSs.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-BSSN_time_evolution-BSSN_RHSs.ipynb to latex
[NbConvertApp] Writing 142774 bytes to Tutorial-BSSN_time_evolution-BSSN_RHSs.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
